# Dice Detection Project
Dice detection project for Computer Vision (include description)

Import dependencies

In [1]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2
import matplotlib.pyplot as plt
from ultralytics.utils.plotting import Annotator

import kagglehub

C:\Users\samal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project constants

In [2]:
# DATASET_PATH = "./dataset-images/"
path = kagglehub.dataset_download("nellbyler/d6-dice")

print("Path to dataset files:", path)
ANNOTATION_PATH = path + "/d6-dice/Annotations"
IMAGE_PATH = path + "/d6-dice/Images"

Path to dataset files: C:\Users\samal\.cache\kagglehub\datasets\nellbyler\d6-dice\versions\3


Example code

In [3]:
# response = requests.get("https://images.unsplash.com/photo-1600880292203-757bb62b4baf?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=2070&q=80")
# image = Image.open(BytesIO(response.content))
# image = np.asarray(image)

# results = model.predict(image)

# # Convert the image to BGR format for OpenCV
# image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# # Draw bounding boxes on the image
# for box in results[0].boxes:
#     x1, y1, x2, y2 = map(int, box.xyxy[0])
#     cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)

# # Convert the image back to RGB format for display
# image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

# # Display the image with bounding boxes
# Image.fromarray(image_rgb)

### Example with dice

Split data into train/test/validation

In [4]:
import os
# import random

anotation_files = []
for dir,_,files in os.walk(ANNOTATION_PATH):
    for filename in files:
        if filename[-4:] == '.txt':
            anotation_files += [(os.path.join(dir,filename))]

# remove classes.txt file from image pool
anotation_files.pop(0)

# 70/15/15 train-test split
train_ratio = .7
test_ratio = (1.0 - train_ratio) / 2
valid_ratio = (1.0 - train_ratio) / 2

SIZE = len(anotation_files)
N = list(range(SIZE))
# random.suffle(N)

train_size = int(SIZE*train_ratio)
test_size = int(SIZE*test_ratio)
valid_size = int(SIZE*valid_ratio)

# get indexes of split list
train_i = N[:train_size]
valid_i = N[train_size:(train_size + valid_size)]
test_i = N[train_size+valid_size:]

print(len(train_i))
print(len(valid_i))
print(len(test_i))
print(len(anotation_files))

175
37
38
250


In [ ]:
import shutil


split_char = '\\'
#split_char = '/'

# Create directories if they don't exist
os.makedirs('datasets/train/images', exist_ok=True)
os.makedirs('datasets/train/labels', exist_ok=True)

os.makedirs('datasets/test/images', exist_ok=True)
os.makedirs('datasets/test/labels', exist_ok=True)

os.makedirs('datasets/valid/images', exist_ok=True)
os.makedirs('datasets/valid/labels', exist_ok=True)

# copy image and label files into local directories
for i in train_i:
    ano_file = anotation_files[i]
    img_file = ano_file.split(split_char)[-1][0:-4] + '.jpg'

    shutil.copy(ano_file, './datasets/train/labels')
    shutil.copy(os.path.join(IMAGE_PATH, img_file), os.path.join('datasets/train/images/', img_file))

for i in test_i:
    ano_file = anotation_files[i]
    img_file = ano_file.split(split_char)[-1][0:-4] + '.jpg'
    
    shutil.copy(ano_file, './datasets/test/labels')
    shutil.copy(os.path.join(IMAGE_PATH, img_file), os.path.join('datasets/test/images/', img_file))

for i in valid_i:
    ano_file = anotation_files[i]
    img_file = ano_file.split(split_char)[-1][0:-4] + '.jpg'

    shutil.copy(ano_file, './datasets/valid/labels')
    shutil.copy(os.path.join(IMAGE_PATH, img_file), os.path.join('datasets/valid/images/', img_file))

In [ ]:
import yaml
source_path = 'C:\\Users\\samal\\Computer-vision-project\\DiceRollDetection'
train_path = 'C:\\Users\\leeal\\OneDrive - Virginia Tech\\School\\Computer Vision\\DiceRollDetection\\datasets\\train'
test_path = 'C:\\Users\\leeal\\OneDrive - Virginia Tech\\School\\Computer Vision\\DiceRollDetection\\datasets\\test'
valid_path = 'C:\\Users\\leeal\\OneDrive - Virginia Tech\\School\\Computer Vision\\DiceRollDetection\\datasets\\valid'

data_yaml = dict(
    train = train_path,
    test = test_path,
    val = valid_path,
    nc = 6,
    names = list('123456')
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

SyntaxError: invalid syntax (341758693.py, line 2)

In [7]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 202E-98F4

 Directory of c:\Users\samal\Computer-vision-project\DiceRollDetection

12/01/2024  10:38 PM    <DIR>          .
12/01/2024  10:36 PM    <DIR>          ..
12/01/2024  10:36 PM             3,365 .gitignore
12/01/2024  10:36 PM             8,207 control_test.ipynb
12/01/2024  10:40 PM               332 data.yaml
12/01/2024  10:37 PM    <DIR>          datasets
12/01/2024  10:36 PM           131,518 dice.png
12/01/2024  10:36 PM            68,290 Dice_Detection.ipynb
12/01/2024  10:38 PM    <DIR>          runs
12/01/2024  10:37 PM         5,613,764 yolo11n.pt
               6 File(s)      5,825,476 bytes
               4 Dir(s)  125,858,332,672 bytes free


In [11]:
names = list(range(1,7))
M=list(range(len(names)))
class_map=dict(zip(M,names))

model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

Transferred 499/499 items from pretrained weights


In [ ]:
train_results = model.train(
    data = "./data.yaml",
    epochs = 14,
    imgsz = 175,
    device = "cpu"
)

New https://pypi.org/project/ultralytics/8.3.39 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.34  Python-3.11.5 torch-2.5.1+cpu CPU (AMD Ryzen 9 5900HS with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolo11n.yaml, data=./data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=175, save=True, save_period=-1, cache=False, device=cpu, workers=0, project=None, name=train1322, exist_ok=False, pretrained=yolo11n.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning C:\Users\leeal\OneDrive - Virginia Tech\School\Computer Vision\DiceRollDetection\datasets\train\labels.cache... 175 images, 0 backgrounds, 0 corrupt: 100%|██████████| 175/175 [00:00<?, ?it/s]
val: Scanning C:\Users\leeal\OneDrive - Virginia Tech\School\Computer Vision\DiceRollDetection\datasets\valid\labels.cache... 37 images, 1 backgrounds, 0 corrupt: 100%|██████████| 38/38 [00:00<?, ?it/s]

Plotting labels to c:\Users\leeal\OneDrive - Virginia Tech\School\Computer Vision\DiceRollDetection\runs\detect\train1322\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 192 train, 192 val
Using 0 dataloader workers
Logging results to c:\Users\leeal\OneDrive - Virginia Tech\School\Computer Vision\DiceRollDetection\runs\detect\train1322
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G       2.21      2.364     0.9168        165        192: 100%|██████████| 11/11 [00:14<00:00,  1.34s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.14s/it]

...

     27/100         0G      1.642      1.836     0.8448        124        192: 100%|██████████| 11/11 [00:10<00:00,  1.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]

                   all         38        203      0.173      0.895      0.225      0.119



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     28/100         0G      1.669      1.845     0.8555        197        192: 100%|██████████| 11/11 [00:09<00:00,  1.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [21]:
trained_model = YOLO("runs/detect/train132/weights/best.pt")
results = trained_model("datasets/train/images/IMG_20191208_111228.jpg")

# Visualize the results
for result in results:
    result.show()

# paths2=[]
# for dirname, _, filenames in os.walk('/kaggle/working/runs/detect/train'):
#     for filename in filenames:
#         if filename[-4:]=='.jpg':
#             paths2+=[(os.path.join(dirname, filename))]
# paths2=sorted(paths2)
# print(len(paths2))


image 1/1 c:\Users\leeal\OneDrive - Virginia Tech\School\Computer Vision\DiceRollDetection\datasets\train\images\IMG_20191208_111228.jpg: 192x192 1 1, 2 2s, 2 3s, 2 5s, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 0.0ms postprocess per image at shape (1, 3, 192, 192)


In [14]:
for path in paths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

In [8]:
# img = Image.open("./dataset-images/000ac425-289e-4ab8-af15-7228b485824e.jpg")
# img = np.asarray(img)

# results = model.predict(img)

# for r in results:
#     annotator = Annotator(img)
    
#     boxes = r.boxes
#     for box in boxes:
        
#         b = box.xyxy[0]  # get box coordinates in (left, top, right, bottom) format
#         c = box.cls
#         annotator.box_label(b, model.names[int(c)])
# img = annotator.result()  
# plt.imshow(img)


# # # Convert the image to BGR format for OpenCV
# # image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# # # Draw bounding boxes on the image
# # for box in results[0].boxes:
# #     x1, y1, x2, y2 = map(int, box.xyxy[0])
# #     cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)

# # # Convert the image back to RGB format for display
# # image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

# # # Display the image with bounding boxes
# # Image.fromarray(image_rgb)